In [ ]:
import os

from cryptography.hazmat.primitives import serialization

In [ ]:
default_key_path = "~/.ssh/magic_packet.pem"
default_key_name = "magic_packet"
default_region = "us-east-1"  # TODO: use aws sdk to retrieve this from config
variables = {
    "public_key": "",
    "key_name": default_key_name,
    "region": default_region,
}

In [ ]:
# Retrieve public key from private key file
key_path = (
    input(f"Enter the RSA private key path (or {default_key_path}): ")
    or default_key_path
)
key_path = os.path.expanduser(key_path)
with open(key_path, "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,
    )
    public_key = private_key.public_key()
    public_bytes = public_key.public_bytes(
        serialization.Encoding.OpenSSH, serialization.PublicFormat.OpenSSH
    )
    variables["public_key"] = public_bytes.decode("utf-8")

In [ ]:
# Provide key name
variables["key_name"] = (
    input(f"Enter the key name (or {default_key_name}): ") or default_key_name
)

In [ ]:
# Provide AWS region
variables["region"] = (
    input(f"Enter the AWS region (or {default_region}): ") or default_region
)

In [ ]:
values = [f'"{key}={value}"' for key, value in variables.items()]
flags = ["-var"] * len(values)
options = " ".join([arg for tup in zip(flags, values) for arg in tup])

In [ ]:
!terraform plan $options

In [ ]:
!terraform apply -auto-approve $options

In [ ]:
!terraform destroy -auto-approve $options